In [ ]:
# Importar las bibliotecas
import pandas as pd
from google.colab import drive
# Montar Google Drive
drive.mount('/content/drive')
# Ruta donde se guardará el archivo en Google Drive
ruta_archivo = '/content/drive/My Drive/Tesis/data/data_tesis_completa_local.xlsx'

Mounted at /content/drive


In [ ]:
!pip install transformers datasets torch
!pip install pytorch-tabnet

## **PREPROCESAMIENTO DE LA DATA**

In [ ]:
df = pd.read_excel(ruta_archivo)

In [ ]:
estadio_alturas = {
  'Estadio Banco Guayaquil': 2518,
  'Estadio Rodrigo Paz Delgado': 2726,
  'Estadio Monumental Isidro Romero Carbo': 9,
  'Estadio Olímpico Atahualpa': 2783,
  'Estadio 9 de Mayo':5,
  'Estadio Federativo Reina del Cisne': 2072,
  'Estadio Jocay':34,
  'Estadio Olímpico de Ibarra': 2200,
  'Estadio Bellavista de Ambato': 2617,
  'Estadio Gonzalo Pozo Ripalda': 2867,
  'Estadio del Mushuc Runa Sporting Club': 3252,
  'Estadio George Capwell':3,
  'Estadio Alejandro Serrano Aguilar': 2523,
  'Estadio Municipal La Cocha': 2778,
  'Estadio Christian Benítez Betancourt':4,
  'Estadio Municipal Jorge Andrade Cantos': 2514,
  'Estadio Gerardo León Pozo': 2296,
  'Estadio Modelo Alberto Spencer':11,
  'Estadio Olímpico de Riobamba': 2771,
  'Estadio General Rumiñahui del Valle': 2500,
  'Estadio Reales Tamarindos':53
}

Equipos_sin_altura =['9 de Octubre FC',
                     'Barcelona SC',
                     'CS Emelec',
                     'Delfin SC',
                     'Fuerza Amarilla SC',
                     'Guayaquil City FC',
                     'LDU Portoviejo',
                     'Manta FC',
                     'Orense SC']

Equipos_con_altura =['CD Independiente del Valle',
                     'LDU de Quito',
                     'CD El Nacional',
                     'Libertad FC',
                     'Imbabura SC',
                     'CD Macara',
                     'SD Aucas',
                     'Mushuc Runa SC',
                     'CD Universidad Catolica',
                     'Cumbaya FC',
                     'CD Tecnico Universitario',
                     'CD Cuenca',
                     'Gualaceo SC',
                     'CD Olmedo',
                     'CD America Quito']

Altura_por_equipo = {
    '9 de Octubre FC': 11,
    'Barcelona SC': 9,
    'CD America Quito': 2783,
    'CD Cuenca':2523,
    'CD El Nacional': 2783,
    'CD Independiente del Valle': 2518,
    'CD Macara': 2617,
    'CD Olmedo': 2771,
    'CD Tecnico Universitario':2617,
    'CD Universidad Catolica': 2783,
    'CS Emelec':3,
    'Cumbaya FC':2783,
    'Delfin SC': 34,
    'Fuerza Amarilla SC': 5,
    'Gualaceo SC': 2296,
    'Guayaquil City FC':4,
    'Imbabura SC': 2200,
    'LDU de Quito': 2726,
    'LDU Portoviejo': 53,
    'Libertad FC': 2072,
    'Manta FC': 34,
    'Mushuc Runa SC': 3252,
    'Orense SC': 5,
    'SD Aucas': 2867,
}

In [ ]:
# Reemplazo de los nombres de los estadios por sus alturas
df['Estadio'] = df['Estadio'].replace(estadio_alturas)
df = df.rename(columns={'Estadio': 'Altura'})

df['Saldo_Goles'] = df['GF'] - df['GC']
df = df.drop(['GF', 'GC'], axis=1)

# Calcular la diferencia de altitud
df['diferencia_altitud'] = df['Altura'] - df['Adversario'].map(Altura_por_equipo)

# Scalar las probabilidades
df['P. Victoria'] = 1 / df['P. Victoria'].astype(float)
df['P. Empate'] = 1 / df['P. Empate'].astype(float)
df['P. Perdida'] = 1 / df['P. Perdida'].astype(float)

<ipython-input-370-86aa82e6d26d>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Estadio'] = df['Estadio'].replace(estadio_alturas)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
encoder = OneHotEncoder(sparse_output=False)
prefijo_vst = 'vst_'
prefijo_lcl = 'lcl_'

# Formaciones del Oponente
formaciones_oponente_encoded = encoder.fit_transform(df[['Formación del oponente']])
formaciones_oponente_df = pd.DataFrame(formaciones_oponente_encoded, columns=[f"{prefijo_vst}{categoria}" for categoria in encoder.categories_[0]])

# Formaciones del Local
formaciones_encoded = encoder.fit_transform(df[['Formación']])
formaciones_df = pd.DataFrame(formaciones_encoded, columns=[f"{prefijo_lcl}{categoria}" for categoria in encoder.categories_[0]])

In [ ]:
# Unir los datos preprocesados
df = pd.concat([df, formaciones_oponente_df,formaciones_df], axis=1)

In [ ]:
def clasificar_partido(row):
    local_en_altura = row['Equipo'] in Equipos_con_altura
    visitante_en_altura = row['Adversario'] in Equipos_con_altura
    local_en_costa = row['Equipo'] in Equipos_sin_altura
    visitante_en_costa = row['Adversario'] in Equipos_sin_altura

    if local_en_altura and visitante_en_altura:
        return 'Altura vs Altura'
    elif local_en_costa and visitante_en_costa:
        return 'Costa vs Costa'
    elif local_en_altura and visitante_en_costa:
        return 'Altura vs Costa'
    elif local_en_costa and visitante_en_altura:
        return 'Costa vs Altura'

# Aplicar la función al DataFrame
df['clasificacion_del_partido'] = df.apply(clasificar_partido, axis=1)

In [ ]:
df_sierra_costa = df[df['clasificacion_del_partido'] == 'Altura vs Costa']
df_costa_sierra = df[df['clasificacion_del_partido'] == 'Costa vs Altura']
df_sierra_sierra = df[df['clasificacion_del_partido'] == 'Altura vs Altura']
df_costa_costa = df[df['clasificacion_del_partido'] == 'Costa vs Costa']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Escalar variables continuas
scaler = MinMaxScaler()
df[[ 'diferencia_altitud', 'Saldo_Goles','Altura']] = scaler.fit_transform( df[['diferencia_altitud', 'Saldo_Goles','Altura']])

## **MODELO DE RED NEURONAL MLP**

In [ ]:
data_evaluar = df.copy()

In [ ]:
y_variables = [
    "vst_3-1-4-2",
    "vst_3-3-3-1",
    "vst_3-4-1-2",
    "vst_3-4-2-1",
    "vst_3-4-3",
    "vst_3-5-1-1",
    "vst_3-5-2",
    "vst_4-1-3-2",
    "vst_4-1-4-1",
    "vst_4-2-2-2",
    "vst_4-2-3-1",
    "vst_4-3-1-2",
    "vst_4-3-2-1",
    "vst_4-3-3",
    "vst_4-4-1-1",
    "vst_4-4-2",
    "vst_4-5-1",
    "vst_5-3-2",
    "vst_5-4-1",]
x_variables = [
    "P. Victoria",
    "P. Empate",
    "P. Perdida",
    "Saldo_Goles",
    "diferencia_altitud",
    "lcl_3-1-4-2",
    "lcl_3-2-4-1",
    "lcl_3-4-1-2",
    "lcl_3-4-3",
    "lcl_3-5-1-1",
    "lcl_3-5-2",
    "lcl_4-1-3-2",
    "lcl_4-1-4-1",
    "lcl_4-2-2-2",
    "lcl_4-2-3-1",
    "lcl_4-3-1-2",
    "lcl_4-3-2-1",
    "lcl_4-3-3",
    "lcl_4-4-1-1",
    "lcl_4-4-2",
    "lcl_4-5-1",
    "lcl_5-3-2",
    "lcl_5-4-1"
]

In [ ]:
val_data = data_evaluar[(data_evaluar['Resultado'] == 'D') & (data_evaluar['Fecha'].dt.year == 2024) &(data_evaluar['clasificacion_del_partido'] == 'Altura vs Costa')]

In [ ]:
test_data = data_evaluar[data_evaluar['Fecha'].dt.year == 2024]
train_data = data_evaluar[data_evaluar['Fecha'].dt.year != 2024]

In [ ]:
# Separar características y etiquetas
X_train = train_data[x_variables]  # Características de entrenamiento
y_train = train_data[y_variables]      # Etiquetas de entrenamiento

X_test = test_data[x_variables]        # Características de validación
y_test = test_data[y_variables]          # Etiquetas de validación

In [ ]:
import tensorflow as tf
import numpy as np

# Función de pérdida ponderada
def weighted_loss(y_true, y_pred):
    # Convertir 'y_true' a formato One-Hot
    y_true_one_hot = tf.one_hot(tf.argmax(y_true, axis=1), depth=y_pred.shape[1])

    # Obtener los índices de los datos del batch actual
    batch_indices = tf.range(tf.shape(y_true)[0])

    # Obtener los valores de 'resultado' y 'altura' para el batch actual
    resultado_batch = tf.gather(df['Resultado'].values, batch_indices)
    #altura_batch = tf.gather(df['altura'].values, batch_indices)

    # Crear pesos para la pérdida para el batch actual
    weights = tf.where(resultado_batch == 'V', 3.0,
                       tf.where(resultado_batch == 'E', 2.0,
                                1.0))

    # Calcular la pérdida ponderada
    loss = tf.reduce_mean(tf.losses.categorical_crossentropy(y_true_one_hot, y_pred) * weights)

    return loss

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
# Crear el modelo de red neuronal

model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')  # Salida con softmax para clasificación
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compilar el modelo con la función de pérdida ponderada
model.compile(optimizer='adam', loss=weighted_loss, metrics=['accuracy'])

In [ ]:
# Entrenar el modelo
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.1296 - loss: 6.5954 - val_accuracy: 0.3636 - val_loss: 5.8056
Epoch 2/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3608 - loss: 5.4641 - val_accuracy: 0.3636 - val_loss: 4.4968
Epoch 3/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3562 - loss: 4.5245 - val_accuracy: 0.3595 - val_loss: 4.2324
Epoch 4/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3638 - loss: 4.3457 - val_accuracy: 0.3636 - val_loss: 4.1943
Epoch 5/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3518 - loss: 4.3267 - val_accuracy: 0.3636 - val_loss: 4.1776
Epoch 6/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3446 - loss: 4.3487 - val_accuracy: 0.3678 - val_loss: 4.1535
Epoch 7/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3452 - loss: 4.3157 - val_accuracy: 0.3719 - val_loss: 4.1665
Epoch 8/100
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3574 - loss: 4.3594 - val_accuracy: 0.3678 - v

In [ ]:
predictions = model.predict(val_data[x_variables])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


In [ ]:
predicted_classes = predictions.argmax(axis=1)

In [ ]:
predicted_classes

array([10, 10, 15, 15, 10, 10, 10, 10, 10, 10, 10, 10, 10])

In [ ]:
predicted_true = np.argmax(val_data[y_variables].values, axis=1)

In [ ]:
predicted_true

array([17, 15, 15,  4, 10, 10, 17, 10, 10, 15, 10,  8,  8])

In [ ]:
# Cálculo de precisión
accuracy = accuracy_score(predicted_true, predicted_classes)
print(f"Precisión: {accuracy:.2f}")

Precisión: 0.46


In [ ]:
all_results = []
for i, row in enumerate(predictions):
    row_results = []
    for formation, prob in zip(y_variables, row):
        row_results.append((formation, round(prob * 100, 2)))  # Convertir probabilidad a porcentaje
    row_results.append((val_data['Formación del oponente'].tolist()[i], 100))
    row_results.sort(key=lambda x: x[1], reverse=True)  # Ordenar de mayor a menor
    all_results.append(row_results)

# Mostrar resultados
for i, row_results in enumerate(all_results):
    print(f"\nResultados para el Partido {i + 1}:")
    for formation, percentage in row_results:
      if percentage == 100:
        print(f"Formacion real del partido: {formation}")
      else:
        formation=formation.replace('vst_','')
        print(f"Formación: {formation}, Probabilidad: {percentage}%")


Resultados para el Partido 1:
Formacion real del partido: 5-3-2
Formación: 4-2-3-1, Probabilidad: 50.14%
Formación: 4-4-2, Probabilidad: 19.37%
Formación: 4-3-3, Probabilidad: 10.36%
Formación: 3-4-3, Probabilidad: 4.59%
Formación: 5-4-1, Probabilidad: 2.77%
Formación: 4-1-4-1, Probabilidad: 2.31%
Formación: 4-3-1-2, Probabilidad: 2.0%
Formación: 3-5-2, Probabilidad: 1.7%
Formación: 4-4-1-1, Probabilidad: 1.62%
Formación: 3-4-1-2, Probabilidad: 1.54%
Formación: 4-2-2-2, Probabilidad: 1.53%
Formación: 3-1-4-2, Probabilidad: 0.63%
Formación: 4-5-1, Probabilidad: 0.53%
Formación: 5-3-2, Probabilidad: 0.39%
Formación: 4-3-2-1, Probabilidad: 0.18%
Formación: 3-4-2-1, Probabilidad: 0.15%
Formación: 4-1-3-2, Probabilidad: 0.11%
Formación: 3-3-3-1, Probabilidad: 0.05%
Formación: 3-5-1-1, Probabilidad: 0.04%

Resultados para el Partido 2:
Formacion real del partido: 4-4-2
Formación: 4-2-3-1, Probabilidad: 46.74%
Formación: 4-4-2, Probabilidad: 21.45%
Formación: 4-3-3, Probabilidad: 10.71%
Form

## **MODELO DE RANDOM FOREST**

In [ ]:
data_RF = df.copy()

In [ ]:
y_variables = "Formación del oponente"

x_variables = [
    "P. Victoria",
    "P. Empate",
    "P. Perdida",
    "Saldo_Goles",
    "diferencia_altitud",
    "lcl_3-1-4-2",
    "lcl_3-2-4-1",
    "lcl_3-4-1-2",
    "lcl_3-4-3",
    "lcl_3-5-1-1",
    "lcl_3-5-2",
    "lcl_4-1-3-2",
    "lcl_4-1-4-1",
    "lcl_4-2-2-2",
    "lcl_4-2-3-1",
    "lcl_4-3-1-2",
    "lcl_4-3-2-1",
    "lcl_4-3-3",
    "lcl_4-4-1-1",
    "lcl_4-4-2",
    "lcl_4-5-1",
    "lcl_5-3-2",
    "lcl_5-4-1"
]

In [ ]:
val_data = data_RF[(data_RF['Resultado'] == 'D') & (data_RF['Fecha'].dt.year == 2024) &(data_RF['clasificacion_del_partido'] == 'Altura vs Costa')]

In [ ]:
test_data = data_RF[data_RF['Fecha'].dt.year == 2024]
train_data = data_RF[data_RF['Fecha'].dt.year != 2024]

In [ ]:
# Asignar las variables de entrada y salida
X_train = train_data[x_variables]
y_train = train_data[y_variables]

X_test = test_data[x_variables]
y_test = test_data[y_variables]


In [ ]:
weights_train = train_data['Resultado'].map({'V': 2, 'E': 1, 'D': 0})
weights_test = test_data['Resultado'].map({'V': 2, 'E': 1, 'D': 0})

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Crear el modelo de Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
# Entrenar el modelo con pesos
rf.fit(X_train, y_train, sample_weight=weights_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_test

,Formación del oponente
0,4-3-3
1,4-2-3-1
2,3-4-3
3,4-4-2
4,4-4-2
...,...
237,5-3-2
238,4-3-3
239,4-1-4-1
240,4-2-3-1


In [ ]:
# Predicción en el conjunto de prueba
y_pred = rf.predict(val_data[x_variables])

In [ ]:
# Obtener las probabilidades de cada clase
probabilities = rf.predict_proba(val_data[x_variables])
# Obtener las clases
classes = rf.classes_

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
# Evaluar el rendimiento del modelo
print("Accuracy:", accuracy_score(val_data[y_variables], y_pred))
print(classification_report(val_data[y_variables], y_pred))

Accuracy: 0.23076923076923078
              precision    recall  f1-score   support

       3-4-3       0.00      0.00      0.00         1
     4-1-4-1       0.00      0.00      0.00         2
     4-2-3-1       0.33      0.60      0.43         5
       4-4-2       0.00      0.00      0.00         3
       5-3-2       0.00      0.00      0.00         2

    accuracy                           0.23        13
   macro avg       0.07      0.12      0.09        13
weighted avg       0.13      0.23      0.16        13



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
resultados = []
for probabilidad in  probabilities:
  lista =[]
  for clase, prob in zip(classes, probabilidad):
    #lista.append((clase, prob*100))
    lista.append((clase, round(prob*100,2)))
  lista.sort(key=lambda x: x[1], reverse=True)
  resultados.append(lista)

In [ ]:
all_results = []
for i, row in enumerate(resultados):
    row_results = []
    for formation, prob in row:  # Iterate through tuples in row
        row_results.append((formation, round(prob, 2)))  # Round the probability value
    #row_results.append((verdaderos_v[i], 100))
    row_results.append((val_data['Formación del oponente'].tolist()[i], 100))
    row_results.sort(key=lambda x: x[1], reverse=True)
    all_results.append(row_results)

# Mostrar resultados
for i, row_results in enumerate(all_results):
    print(f"\nResultados para el Partido {i + 1}:")
    for formation, percentage in row_results:
      if percentage == 100:
        print(f"Formacion real del partido: {formation}")
      else:
        formation=formation.replace('vst_','')
        print(f"Formación: {formation}, Probabilidad: {percentage}%")


Resultados para el Partido 1:
Formacion real del partido: 5-3-2
Formación: 4-2-3-1, Probabilidad: 68.0%
Formación: 4-4-2, Probabilidad: 19.0%
Formación: 3-4-3, Probabilidad: 4.0%
Formación: 4-3-3, Probabilidad: 4.0%
Formación: 5-4-1, Probabilidad: 2.0%
Formación: 3-1-4-2, Probabilidad: 1.0%
Formación: 4-1-4-1, Probabilidad: 1.0%
Formación: 4-4-1-1, Probabilidad: 1.0%
Formación: 3-3-3-1, Probabilidad: 0.0%
Formación: 3-4-1-2, Probabilidad: 0.0%
Formación: 3-4-2-1, Probabilidad: 0.0%
Formación: 3-5-1-1, Probabilidad: 0.0%
Formación: 3-5-2, Probabilidad: 0.0%
Formación: 4-1-3-2, Probabilidad: 0.0%
Formación: 4-2-2-2, Probabilidad: 0.0%
Formación: 4-3-1-2, Probabilidad: 0.0%
Formación: 4-3-2-1, Probabilidad: 0.0%
Formación: 4-5-1, Probabilidad: 0.0%
Formación: 5-3-2, Probabilidad: 0.0%

Resultados para el Partido 2:
Formacion real del partido: 4-4-2
Formación: 4-2-3-1, Probabilidad: 65.0%
Formación: 4-4-2, Probabilidad: 21.0%
Formación: 3-4-3, Probabilidad: 4.0%
Formación: 5-4-1, Probabil

## **MODELO DE SVM**

In [ ]:
data_SVM = df.copy()

In [ ]:
y_variables = "Formación del oponente"

x_variables = [
    "P. Victoria",
    "P. Empate",
    "P. Perdida",
    "Saldo_Goles",
    "diferencia_altitud",
    "lcl_3-1-4-2",
    "lcl_3-2-4-1",
    "lcl_3-4-1-2",
    "lcl_3-4-3",
    "lcl_3-5-1-1",
    "lcl_3-5-2",
    "lcl_4-1-3-2",
    "lcl_4-1-4-1",
    "lcl_4-2-2-2",
    "lcl_4-2-3-1",
    "lcl_4-3-1-2",
    "lcl_4-3-2-1",
    "lcl_4-3-3",
    "lcl_4-4-1-1",
    "lcl_4-4-2",
    "lcl_4-5-1",
    "lcl_5-3-2",
    "lcl_5-4-1"
]

In [ ]:
val_data = data_SVM[(data_SVM['Resultado'] == 'D') & (data_SVM['Fecha'].dt.year == 2024) &(data_SVM['clasificacion_del_partido'] == 'Altura vs Costa')]

In [ ]:
test_data = data_SVM[data_SVM['Fecha'].dt.year == 2024]
train_data = data_SVM[data_SVM['Fecha'].dt.year != 2024]

In [ ]:
# Asignar las variables de entrada y salida
X_train = train_data[x_variables]
y_train = train_data[y_variables]

X_test = test_data[x_variables]
y_test = test_data[y_variables]

In [ ]:
sample_weights = train_data['Resultado'].map({'V': 3, 'E': 1, 'D': -3})

In [ ]:
from sklearn.svm import SVC

# Entrenamiento del modelo SVM
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', probability=True)  # OvR para multiclase
svm_model.fit(X_train, y_train, sample_weight=sample_weights)

SVC(probability=True)

In [ ]:
# Predicción y evaluación
y_pred = svm_model.predict(val_data[x_variables])

In [ ]:
probabilities = svm_model.predict_proba(val_data[x_variables])

In [ ]:
classes = svm_model.classes_

In [ ]:
resultados = []
for probabilidad in  probabilities:
  lista =[]
  for clase, prob in zip(classes, probabilidad):
    #lista.append((clase, prob*100))
    lista.append((clase, round(prob*100,2)))
  lista.sort(key=lambda x: x[1], reverse=True)
  resultados.append(lista)

In [ ]:
all_results = []
for i, row in enumerate(resultados):
    row_results = []
    for formation, prob in row:  # Iterate through tuples in row
        row_results.append((formation, round(prob, 2)))  # Round the probability value
    #row_results.append((verdaderos_v[i], 100))
    row_results.append((val_data['Formación del oponente'].tolist()[i], 100))
    row_results.sort(key=lambda x: x[1], reverse=True)
    all_results.append(row_results)

# Mostrar resultados
for i, row_results in enumerate(all_results):
    print(f"\nResultados para el Partido {i + 1}:")
    for formation, percentage in row_results:
      if percentage == 100:
        print(f"Formacion real del partido: {formation}")
      else:
        formation=formation.replace('vst_','')
        print(f"Formación: {formation}, Probabilidad: {percentage}%")


Resultados para el Partido 1:
Formacion real del partido: 5-3-2
Formación: 4-2-3-1, Probabilidad: 34.49%
Formación: 4-4-2, Probabilidad: 25.28%
Formación: 4-3-3, Probabilidad: 14.7%
Formación: 3-4-3, Probabilidad: 7.84%
Formación: 5-3-2, Probabilidad: 3.56%
Formación: 4-1-4-1, Probabilidad: 3.15%
Formación: 3-5-2, Probabilidad: 2.86%
Formación: 5-4-1, Probabilidad: 2.53%
Formación: 4-4-1-1, Probabilidad: 1.19%
Formación: 4-5-1, Probabilidad: 1.16%
Formación: 4-2-2-2, Probabilidad: 0.91%
Formación: 4-3-2-1, Probabilidad: 0.47%
Formación: 3-4-1-2, Probabilidad: 0.44%
Formación: 4-3-1-2, Probabilidad: 0.37%
Formación: 4-1-3-2, Probabilidad: 0.28%
Formación: 3-1-4-2, Probabilidad: 0.27%
Formación: 3-5-1-1, Probabilidad: 0.18%
Formación: 3-3-3-1, Probabilidad: 0.17%
Formación: 3-4-2-1, Probabilidad: 0.15%

Resultados para el Partido 2:
Formacion real del partido: 4-4-2
Formación: 4-2-3-1, Probabilidad: 34.46%
Formación: 4-4-2, Probabilidad: 25.31%
Formación: 4-3-3, Probabilidad: 14.65%
For

In [ ]:
print("Precisión del modelo:", accuracy_score(val_data[y_variables], y_pred))
print("Reporte de clasificación:")
print(classification_report(val_data[y_variables], y_pred))

Precisión del modelo: 0.38461538461538464
Reporte de clasificación:
              precision    recall  f1-score   support

       3-4-3       0.00      0.00      0.00         1
     4-1-4-1       0.00      0.00      0.00         2
     4-2-3-1       0.42      1.00      0.59         5
       4-4-2       0.00      0.00      0.00         3
       5-3-2       0.00      0.00      0.00         2

    accuracy                           0.38        13
   macro avg       0.08      0.20      0.12        13
weighted avg       0.16      0.38      0.23        13



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## **MODELO DE TABNET**

In [ ]:
data_TBN = df.copy()

NameError: name 'df' is not defined

In [ ]:
y_variables = "Formación del oponente"

x_variables = [
    "P. Victoria",
    "P. Empate",
    "P. Perdida",
    "Saldo_Goles",
    "diferencia_altitud",
    "lcl_3-1-4-2",
    "lcl_3-2-4-1",
    "lcl_3-4-1-2",
    "lcl_3-4-3",
    "lcl_3-5-1-1",
    "lcl_3-5-2",
    "lcl_4-1-3-2",
    "lcl_4-1-4-1",
    "lcl_4-2-2-2",
    "lcl_4-2-3-1",
    "lcl_4-3-1-2",
    "lcl_4-3-2-1",
    "lcl_4-3-3",
    "lcl_4-4-1-1",
    "lcl_4-4-2",
    "lcl_4-5-1",
    "lcl_5-3-2",
    "lcl_5-4-1"
]

In [ ]:
# Lista de formaciones a eliminar
formaciones_a_filtrar = ['3-3-3-1', '3-4-2-1', '3-5-1-1', '4-3-2-1', '3-1-4-2']

# Eliminar filas donde 'Formación del oponente' esté en la lista
data_TBN = data_TBN[~data_TBN['Formación del oponente'].isin(formaciones_a_filtrar)]

# Mostrar el DataFrame resultante
print(data_TBN)

NameError: name 'data_TBN' is not defined

In [ ]:
val_data = data_TBN[(data_TBN['Resultado'] == 'D') & (data_TBN['Fecha'].dt.year == 2024) &(data_TBN['clasificacion_del_partido'] == 'Altura vs Costa')]

In [ ]:
test_data = data_TBN[data_TBN['Fecha'].dt.year == 2024]
train_data = data_TBN[data_TBN['Fecha'].dt.year != 2024]

In [ ]:
# Asignar las variables de entrada y salida
X_train = train_data[x_variables]
y_train = train_data[y_variables]

X_test = test_data[x_variables]
y_test = test_data[y_variables]

In [ ]:
print("Clases en y_true:", np.unique(y_train))
print("Clases en y_pred:", np.unique(y_test))


In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np


# Definir y entrenar TabNet
clf = TabNetClassifier()

pesos = np.where(
    train_data['Resultado'] == 'V', 3.0,  # Peso para victorias
    np.where(train_data['Resultado'] == 'E', 2.0, 1.0)  # Peso para empates y derrotas
)

clf.fit(
    X_train.values, y_train.values,
    eval_set=[(X_test.values, y_test.values)],
    eval_name=['val'],
    eval_metric=['accuracy', 'logloss'],
    max_epochs=250,
    patience=85,
    weights=pesos,
    batch_size=256,
    virtual_batch_size=128
)

In [ ]:
y_pred = clf.predict(val_data[x_variables].values)  # Get predictions
accuracy = accuracy_score(val_data[y_variables].values, y_pred)  # Calculate accuracy
print(f"Precisión del modelo: {accuracy}")

In [ ]:
probabilidades = clf.predict_proba(val_data[x_variables].values)
clf.classes_

In [ ]:
resultados = []
for probabilidad in  probabilidades:
  lista =[]
  for clase, prob in zip(clf.classes_, probabilidad): # Use probabilidad instead of probabilidades
    #lista.append((clase, prob*100))
    lista.append((clase, round(prob*100,2)))
  lista.sort(key=lambda x: x[1], reverse=True)
  resultados.append(lista)

In [ ]:
all_results = []
for i, row in enumerate(resultados):
    row_results = []
    for formation, prob in row:  # Iterate through tuples in row
        row_results.append((formation, round(prob, 2)))  # Round the probability value
    #row_results.append((verdaderos_v[i], 100))
    row_results.append((val_data['Formación del oponente'].tolist()[i], 100))
    row_results.sort(key=lambda x: x[1], reverse=True)
    all_results.append(row_results)

# Mostrar resultados
for i, row_results in enumerate(all_results):
    print(f"\nResultados para el Partido {i + 1}:")
    for formation, percentage in row_results:
      if percentage == 100:
        print(f"Formacion real del partido: {formation}")
      else:
        formation=formation.replace('vst_','')
        print(f"Formación: {formation}, Probabilidad: {percentage}%")